In [6]:
# 대표적인 비지도(Unsupervised) 학습 방법인 Autoencoder 를 구현해봅니다.
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./mnist/data/", one_hot=True)

Extracting ./mnist/data/train-images-idx3-ubyte.gz
Extracting ./mnist/data/train-labels-idx1-ubyte.gz
Extracting ./mnist/data/t10k-images-idx3-ubyte.gz
Extracting ./mnist/data/t10k-labels-idx1-ubyte.gz


In [13]:
#########
# 옵션 설정

learning_rate = 0.01
training_epoch = 30
batch_size = 100

# 신경망 레이어 구성 옵션
n_hidden = 256  
n_hidden2 = 128 
n_hidden3 = 64 
n_input = 28*28   

In [14]:
#########
# 신경망 모델 구성
######
# Y 가 없습니다. 입력값을 Y로 사용하기 때문입니다.
X = tf.placeholder(tf.float32, [None, n_input])

# 인코더 레이어와 디코더 레이어의 가중치와 편향 변수를 설정합니다.
# 다음과 같이 이어지는 레이어를 구성하기 위한 값들 입니다.
# input -> encode -> decode -> output

W_encode = tf.Variable(tf.random_normal([n_input, n_hidden]))
b_encode = tf.Variable(tf.random_normal([n_hidden]))
encoder = tf.nn.sigmoid(
                tf.add(tf.matmul(X, W_encode), b_encode))

W_encode2 = tf.Variable(tf.random_normal([n_hidden, n_hidden2]))
b_encode2 = tf.Variable(tf.random_normal([n_hidden2]))
encoder2 = tf.nn.sigmoid(
                tf.add(tf.matmul(encoder, W_encode2), b_encode2))

W_encode3 = tf.Variable(tf.random_normal([n_hidden2, n_hidden3]))
b_encode3 = tf.Variable(tf.random_normal([n_hidden3]))
encoder3 = tf.nn.sigmoid(
                tf.add(tf.matmul(encoder2, W_encode3), b_encode3))

W_decode = tf.Variable(tf.random_normal([n_hidden3, n_hidden2]))
b_decode = tf.Variable(tf.random_normal([n_hidden2]))
decoder = tf.nn.sigmoid(
                tf.add(tf.matmul(encoder3, W_decode), b_decode))

W_decode2 = tf.Variable(tf.random_normal([n_hidden2, n_hidden]))
b_decode2 = tf.Variable(tf.random_normal([n_hidden]))
decoder2 = tf.nn.sigmoid(
                tf.add(tf.matmul(encoder2, W_decode2), b_decode2))

W_decode3 = tf.Variable(tf.random_normal([n_hidden, n_input]))
b_decode3 = tf.Variable(tf.random_normal([n_input]))
decoder3 = tf.nn.sigmoid(
                tf.add(tf.matmul(encoder, W_decode3), b_decode3))

# 디코더는 인풋과 최대한 같은 결과를 내야 하므로, 디코딩한 결과를 평가하기 위해
# 입력 값인 X 값을 평가를 위한 실측 결과 값으로하여 decoder 와의 차이를 손실값으로 설정합니다.
cost = tf.reduce_mean(tf.pow(X - decoder3, 2))
optimizer = tf.train.RMSPropOptimizer(learning_rate).minimize(cost)

In [ ]:
#########
# 신경망 모델 학습
######
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

total_batch = int(mnist.train.num_examples/batch_size)

for epoch in range(training_epoch):
    total_cost = 0

    for i in range(total_batch):
        batch_xs, batch_ys = mnist.train.next_batch(batch_size)
        _, cost_val = sess.run([optimizer, cost],
                               feed_dict={X: batch_xs})
        total_cost += cost_val

    print('Epoch:', '%04d' % (epoch + 1),
          'Avg. cost =', '{:.4f}'.format(total_cost / total_batch))

print('최적화 완료!')

In [ ]:
#########
# 결과 확인
# 입력값(위쪽)과 모델이 생성한 값(아래쪽)을 시각적으로 비교해봅니다.
######
sample_size = 10

samples = sess.run(decoder3,
                   feed_dict={X: mnist.test.images[:sample_size]})

fig, ax = plt.subplots(2, sample_size, figsize=(sample_size, 2))

for i in range(sample_size):
    ax[0][i].set_axis_off()
    ax[1][i].set_axis_off()
    ax[0][i].imshow(np.reshape(mnist.test.images[i], (28, 28)))
    ax[1][i].imshow(np.reshape(samples[i], (28, 28)))

plt.show()
